In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.io as scio
import time
import matplotlib
import torch.nn as nn
import torch.nn.functional as F

**MOTSP**

In [2]:
class TSPDataset(Dataset):

    def __init__(self, size=50, num_samples=1e6, seed=None):
        super(TSPDataset, self).__init__()

        if seed is None:
            seed = np.random.randint(123456789)

        np.random.seed(seed)
        torch.manual_seed(seed)
        self.dataset = torch.rand((num_samples, 4, size))
        self.dynamic = torch.zeros(num_samples, 1, size)
        self.num_nodes = size
        self.size = num_samples


    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        # (static, dynamic, start_loc)
        return (self.dataset[idx], self.dynamic[idx], [])


def update_mask(mask, dynamic, chosen_idx):
    """Marks the visited city, so it can't be selected a second time."""
    mask.scatter_(1, chosen_idx.unsqueeze(1), 0)
    return mask


def reward(static, tour_indices, w1=1, w2=0):
    """
    Parameters
    ----------
    static: torch.FloatTensor containing static (e.g. x, y) data
    tour_indices: torch.IntTensor of size (batch_size, num_cities)

    Returns
    -------
    Euclidean distance between consecutive nodes on the route. of size
    (batch_size, num_cities)
    """

    # Convert the indices back into a tour
    idx = tour_indices.unsqueeze(1).expand_as(static)
    tour = torch.gather(static.data, 2, idx).permute(0, 2, 1)

    # Make a full tour by returning to the start
    y = torch.cat((tour, tour[:, :1]), dim=1)
    # first 2 is xy coordinate, third column is another obj
    y_dis = y[:, :, :2]
    y_dis2 = y[:, :, 2:]

    # Euclidean distance between each consecutive point
    tour_len = torch.sqrt(torch.sum(torch.pow(y_dis[:, :-1] - y_dis[:, 1:], 2), dim=2))
    obj1 = tour_len.sum(1).detach()

    tour_len2 = torch.sqrt(torch.sum(torch.pow(y_dis2[:, :-1] - y_dis2[:, 1:], 2), dim=2))
    obj2 = tour_len2.sum(1).detach()

    obj = w1*obj1 + w2*obj2
    return obj, obj1, obj2



def render(static, tour_indices, save_path):
    """Plots the found tours."""

    plt.close('all')

    num_plots = 3 if int(np.sqrt(len(tour_indices))) >= 3 else 1

    _, axes = plt.subplots(nrows=num_plots, ncols=num_plots,
                           sharex='col', sharey='row')

    if num_plots == 1:
        axes = [[axes]]
    axes = [a for ax in axes for a in ax]

    for i, ax in enumerate(axes):

        # Convert the indices back into a tour
        idx = tour_indices[i]
        if len(idx.size()) == 1:
            idx = idx.unsqueeze(0)

        # End tour at the starting index
        idx = idx.expand(static.size(1), -1)
        idx = torch.cat((idx, idx[:, 0:1]), dim=1)

        data = torch.gather(static[i].data, 1, idx).cpu().numpy()

        #plt.subplot(num_plots, num_plots, i + 1)
        ax.plot(data[0], data[1], zorder=1)
        ax.scatter(data[0], data[1], s=4, c='r', zorder=2)
        ax.scatter(data[0, 0], data[1, 0], s=20, c='k', marker='*', zorder=3)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)

    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight', dpi=400)

**ENCODER**

In [3]:
class Encoder(nn.Module):
    """Encodes the static & dynamic states using 1d Convolution."""

    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.conv = nn.Conv1d(input_size, hidden_size, kernel_size=1)

    def forward(self, input):
        output = self.conv(input)
        return output  # (batch, hidden_size, seq_len)

**Attention**

In [4]:
class Attention(nn.Module):
    """Calculates attention over the input nodes given the current state."""

    def __init__(self, hidden_size):
        super(Attention, self).__init__()

        # W processes features from static decoder elements
        self.v = nn.Parameter(torch.zeros((1, 1, hidden_size),
                                          device=device, requires_grad=True))

        self.W = nn.Parameter(torch.zeros((1, hidden_size, 3 * hidden_size),
                                          device=device, requires_grad=True))

    def forward(self, static_hidden, dynamic_hidden, decoder_hidden):

        batch_size, hidden_size, _ = static_hidden.size()

        hidden = decoder_hidden.unsqueeze(2).expand_as(static_hidden)
        hidden = torch.cat((static_hidden, dynamic_hidden, hidden), 1)

        # Broadcast some dimensions so we can do batch-matrix-multiply
        v = self.v.expand(batch_size, 1, hidden_size)
        W = self.W.expand(batch_size, hidden_size, -1)

        attns = torch.bmm(v, torch.tanh(torch.bmm(W, hidden)))
        attns = F.softmax(attns, dim=2)  # (batch, seq_len)
        return attns

**Pointer**

In [5]:
class Pointer(nn.Module):
    """Calculates the next state given the previous state and input embeddings."""

    def __init__(self, hidden_size, num_layers=1, dropout=0.2):
        super(Pointer, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Used to calculate probability of selecting next state
        self.v = nn.Parameter(torch.zeros((1, 1, hidden_size),
                                          device=device, requires_grad=True))

        self.W = nn.Parameter(torch.zeros((1, hidden_size, 2 * hidden_size),
                                          device=device, requires_grad=True))

        # Used to compute a representation of the current decoder output
        # GRU（输入dim，隐含层dim，层数）
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers,
                          batch_first=True,
                          dropout=dropout if num_layers > 1 else 0)
        self.encoder_attn = Attention(hidden_size)

        self.drop_rnn = nn.Dropout(p=dropout)
        self.drop_hh = nn.Dropout(p=dropout)

    def forward(self, static_hidden, dynamic_hidden, decoder_hidden, last_hh):

        rnn_out, last_hh = self.gru(decoder_hidden.transpose(2, 1), last_hh)
        rnn_out = rnn_out.squeeze(1)

        # Always apply dropout on the RNN output
        rnn_out = self.drop_rnn(rnn_out)
        if self.num_layers == 1:
            # If > 1 layer dropout is already applied
            last_hh = self.drop_hh(last_hh) 

        # Given a summary of the output, find an  input context
        enc_attn = self.encoder_attn(static_hidden, dynamic_hidden, rnn_out)
        context = enc_attn.bmm(static_hidden.permute(0, 2, 1))  # (B, 1, num_feats)

        # Calculate the next output using Batch-matrix-multiply ops
        context = context.transpose(1, 2).expand_as(static_hidden)
        energy = torch.cat((static_hidden, context), dim=1)  # (B, num_feats, seq_len)

        v = self.v.expand(static_hidden.size(0), -1, -1)
        W = self.W.expand(static_hidden.size(0), -1, -1)

        probs = torch.bmm(v, torch.tanh(torch.bmm(W, energy))).squeeze(1)

        return probs, last_hh

**DRL_4TSP**

In [6]:
class DRL4TSP(nn.Module): 
    """Defines the main Encoder, Decoder, and Pointer combinatorial models.
​
    Parameters
    ----------
    static_size: int
        Defines how many features are in the static elements of the model
        (e.g. 2 for (x, y) coordinates)
    dynamic_size: int > 1
        Defines how many features are in the dynamic elements of the model
        (e.g. 2 for the VRP which has (load, demand) attributes. The TSP doesn't
        have dynamic elements, but to ensure compatility with other optimization
        problems, assume we just pass in a vector of zeros.
    hidden_size: int
        Defines the number of units in the hidden layer for all static, dynamic,
        and decoder output units.
    update_fn: function or None
        If provided, this method is used to calculate how the input dynamic
        elements are updated, and is called after each 'point' to the input element.
    mask_fn: function or None
        Allows us to specify which elements of the input sequence are allowed to
        be selected. This is useful for speeding up training of the networks,
        by providing a sort of 'rules' guidlines to the algorithm. If no mask
        is provided, we terminate the search after a fixed number of iterations
        to avoid tours that stretch forever
    num_layers: int
        Specifies the number of hidden layers to use in the decoder RNN
    dropout: float
        Defines the dropout rate for the decoder
    """
​
    def __init__(self, static_size, dynamic_size, hidden_size,
                 update_fn=None, mask_fn=None, num_layers=1, dropout=0.):
        super(DRL4TSP, self).__init__()
​
        if dynamic_size < 1:
            raise ValueError(':param dynamic_size: must be > 0, even if the '
                             'problem has no dynamic elements')
​
        self.update_fn = update_fn
        self.mask_fn = mask_fn
​
        # Define the encoder & decoder models
        self.static_encoder = Encoder(static_size, hidden_size)
        self.dynamic_encoder = Encoder(dynamic_size, hidden_size)
        self.decoder = Encoder(static_size, hidden_size)
        self.pointer = Pointer(hidden_size, num_layers, dropout)
​
        for p in self.parameters():
            if len(p.shape) > 1:
                nn.init.xavier_uniform_(p)
​
        # Used as a proxy initial state in the decoder when not specified
        self.x0 = torch.zeros((1, static_size, 1), requires_grad=True, device=device)
​
    def forward(self, static, dynamic, decoder_input=None, last_hh=None):
        """
        Parameters
        ----------
        static: Array of size (batch_size, feats, num_cities)
            Defines the elements to consider as static. For the TSP, this could be
            things like the (x, y) coordinates, which won't change
        dynamic: Array of size (batch_size, feats, num_cities)
            Defines the elements to consider as static. For the VRP, this can be
            things like the (load, demand) of each city. If there are no dynamic
            elements, this can be set to None
        decoder_input: Array of size (batch_size, num_feats)
            Defines the outputs for the decoder. Currently, we just use the
            static elements (e.g. (x, y) coordinates), but this can technically
            be other things as well
        last_hh: Array of size (batch_size, num_hidden)
            Defines the last hidden state for the RNN
        """
​
        batch_size, input_size, sequence_size = static.size()
​
        if decoder_input is None:
            decoder_input = self.x0.expand(batch_size, -1, -1)
​
        # Always use a mask - if no function is provided, we don't update it
        mask = torch.ones(batch_size, sequence_size, device=device)
​
        # Structures for holding the output sequences
        tour_idx, tour_logp = [], []
        max_steps = sequence_size if self.mask_fn is None else 1000
​
        # Static elements only need to be processed once, and can be used across
        # all 'pointing' iterations. When / if the dynamic elements change,
        # their representations will need to get calculated again.
        static_hidden = self.static_encoder(static)
        dynamic_hidden = self.dynamic_encoder(dynamic)
​
        for _ in range(max_steps):
​
            if not mask.byte().any():
                break
​
            # ... but compute a hidden rep for each element added to sequence
            decoder_hidden = self.decoder(decoder_input)
​
            probs, last_hh = self.pointer(static_hidden,
                                          dynamic_hidden,
                                          decoder_hidden, last_hh)
            probs = F.softmax(probs + mask.log(), dim=1)
​
            # When training, sample the next step according to its probability.
            # During testing, we can take the greedy approach and choose highest
            if self.training:
                m = torch.distributions.Categorical(probs)
​
                # Sometimes an issue with Categorical & sampling on GPU; See:
                # https://github.com/pemami4911/neural-combinatorial-rl-pytorch/issues/5
                ptr = m.sample()
                while not torch.gather(mask, 1, ptr.data.unsqueeze(1)).byte().all():
                    ptr = m.sample()
                logp = m.log_prob(ptr)
            else:
                prob, ptr = torch.max(probs, 1)  # Greedy
                logp = prob.log()
​
            # After visiting a node update the dynamic representation
            if self.update_fn is not None:
                dynamic = self.update_fn(dynamic, ptr.data)
                dynamic_hidden = self.dynamic_encoder(dynamic)
​
                # Since we compute the VRP in minibatches, some tours may have
                # number of stops. We force the vehicles to remain at the depot 
                # in these cases, and logp := 0
                is_done = dynamic[:, 1].sum(1).eq(0).float()
                logp = logp * (1. - is_done)
​
            # And update the mask so we don't re-visit if we don't need to
            if self.mask_fn is not None:
                mask = self.mask_fn(mask, dynamic, ptr.data).detach()
​
            tour_logp.append(logp.unsqueeze(1))
            tour_idx.append(ptr.data.unsqueeze(1))
​
            decoder_input = torch.gather(static, 2,
                                         ptr.view(-1, 1, 1)
                                         .expand(-1, input_size, 1)).detach()
​
        tour_idx = torch.cat(tour_idx, dim=1)  # (batch_size, seq_len)
        tour_logp = torch.cat(tour_logp, dim=1)  # (batch_size, seq_len)
​
        return tour_idx, tour_logp

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 117)

**StateCritic**

In [ ]:
class StateCritic(nn.Module): 
    """Estimates the problem complexity.

    This is a basic module that just looks at the log-probabilities predicted by
    the encoder + decoder, and returns an estimate of complexity
    """

    def __init__(self, static_size, dynamic_size, hidden_size):
        super(StateCritic, self).__init__()

        self.static_encoder = Encoder(static_size, hidden_size)
        self.dynamic_encoder = Encoder(dynamic_size, hidden_size)

        # Define the encoder & decoder models
        self.fc1 = nn.Conv1d(hidden_size * 2, 20, kernel_size=1)
        self.fc2 = nn.Conv1d(20, 20, kernel_size=1)
        self.fc3 = nn.Conv1d(20, 1, kernel_size=1)

        for p in self.parameters():
            if len(p.shape) > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, static, dynamic):

        # Use the probabilities of visiting each
        static_hidden = self.static_encoder(static)
        dynamic_hidden = self.dynamic_encoder(dynamic)

        hidden = torch.cat((static_hidden, dynamic_hidden), 1)

        output = F.relu(self.fc1(hidden))
        output = F.relu(self.fc2(output))
        output = self.fc3(output).sum(dim=2)
        return output

**Dis_matrix**

In [ ]:
def dis_matrix(static, s_size):
    static = static.squeeze(0) 

    # [2,20]
    obj1 = static[:2, :]
    # [20]
    obj2 = static[2:, :]

    l = obj1.size()[1]
    obj1_matrix = np.zeros((l, l))
    obj2_matrix = np.zeros((l, l))
    for i in range(l):
        for j in range(l):
            if i != j:
                obj1_matrix[i,j] = torch.sqrt(torch.sum(torch.pow(obj1[:, i] - obj1[:, j], 2))).detach()
                if s_size == 3:
                    obj2_matrix[i, j] = torch.abs(obj2[i] - obj2[j]).detach()
                else:
                    obj2_matrix[i, j] = torch.sqrt(torch.sum(torch.pow(obj2[:, i] - obj2[:, j], 2))).detach()

    return obj1_matrix, obj2_matrix

In [ ]:
def update_mask(mask, dynamic, chosen_idx):
    """Marks the visited city, so it can't be selected a second time."""
    mask.scatter_(1, chosen_idx.unsqueeze(1), 0)
    return mask

In [ ]:
class Kro_dataset(Dataset):

    def __init__(self, num_nodes):
        super(Kro_dataset, self).__init__()

        x1 = np.loadtxt('/kaggle/input/mo-tsp1/krodata/kroA%d.tsp'%num_nodes, skiprows=6, usecols=(1, 2), delimiter=' ', dtype=float)
        x1 = x1 / (np.max(x1,0))
        x2 = np.loadtxt('/kaggle/input/mo-tsp1/krodata/kroB%d.tsp'%num_nodes, skiprows=6, usecols=(1, 2), delimiter=' ', dtype=float)
        x2 = x2 / (np.max(x2,0))
        x = np.concatenate((x1, x2),axis=1)
        x = x.T
        x = x.reshape(1, 4, num_nodes)

        self.dataset = torch.from_numpy(x).float()
        self.dynamic = torch.zeros(1, 1, num_nodes)
        self.num_nodes = num_nodes
        self.size = 1


    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        # (static, dynamic, start_loc)
        return (self.dataset[idx], self.dynamic[idx], [])

In [ ]:
# Load the trained model and convert the obtained Pareto Front to the .mat file.
# It is convenient to visualize it in matlab    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
save_dir = "/kaggle/input/mo-tsp3/tsp_transfer_100run_500000_5epoch_40city/40"


update_fn = None
STATIC_SIZE = 4  # (x, y)
DYNAMIC_SIZE = 1  # dummy for compatibility

# claim model
actor = DRL4TSP(STATIC_SIZE,
                DYNAMIC_SIZE,
                128,
                update_fn,
                update_mask,
                1,
                0.1).to(device)
critic = StateCritic(STATIC_SIZE, DYNAMIC_SIZE, 128).to(device)

# data 143
# from Post_process.convet_kro_dataloader import Kro_dataset
kro = 1
D = 200
if kro:
    D = 200
    Test_data = Kro_dataset(D)
    Test_loader = DataLoader(Test_data, 1, False, num_workers=0)
else:
    # 40city_train: city20 13 city40 143 city70 2523
    #
    Test_data = TSPDataset(D, 1, 2523)
    Test_loader = DataLoader(Test_data, 1, False, num_workers=0)

iter_data = iter(Test_loader)
static, dynamic, x0 = next(iter_data)
static = static.to(device)
dynamic = dynamic.to(device)
x0 = x0.to(device) if len(x0) > 0 else None

# load 50 models
N=100
w = np.arange(N+1)/N
objs = np.zeros((N+1,2))
start  = time.time()
t1_all = 0
t2_all = 0
tours=[]
for i in range(0, N+1):
    t1 = time.time()
    ac = os.path.join(save_dir, "w_%2.2f_%2.2f" % (1-w[i], w[i]),"actor.pt")
    cri = os.path.join(save_dir, "w_%2.2f_%2.2f" % (1-w[i], w[i]),"critic.pt")
    actor.load_state_dict(torch.load(ac, device))
    critic.load_state_dict(torch.load(cri, device))
    t1_all = t1_all + time.time()-t1
    # calculate

    with torch.no_grad():
        # t2 = time.time()
        tour_indices, _ = actor.forward(static, dynamic, x0)
        # t2_all = t2_all + time.time() - t2
    _, obj1, obj2 = reward(static, tour_indices, 1-w[i], w[i])
#     tours.append(tour_indices.cpu().numpy())
#     objs[i,:] = [obj1, obj2]
    tour_indices = tour_indices.cpu()
    tours.append(tour_indices.numpy())
    obj1 = obj1.cpu().item()
    obj2 = obj2.cpu().item()
    objs[i,:] = [obj1, obj2]

print("time_load_model:%2.4f"%t1_all)
print("time_predict_model:%2.4f"%t2_all)
print(time.time()-start)

print(tours)
plt.figure()
plt.plot(objs[:,0],objs[:,1],"ro")
plt.show()

# Convert to .mat
obj1_matrix, obj2_matrix = dis_matrix(static, STATIC_SIZE)
print(obj1_matrix)
print(obj2_matrix)

**NSGA-II**

In [ ]:
import random
import math
import matplotlib.pyplot as plt

In [ ]:
distance_matrix = obj1_matrix
cost_matrix = obj2_matrix


def Evaluate(tour, matrix):
    value = 0
    i = 0
    while i < (len(tour)-1):
        value += matrix[tour[i]][tour[i+1]]
        i += 1
        pass
    value += matrix[tour[-1]][tour[0]]
    return value

def CheckDomination(solution_1, solution_2):
    solution_1_cost = Evaluate(solution_1, cost_matrix)
    solution_1_dist = Evaluate(solution_1, distance_matrix)
    solution_2_cost = Evaluate(solution_2, cost_matrix)
    solution_2_dist = Evaluate(solution_2, distance_matrix)

    dist_comp_eql_p = solution_1_dist <= solution_2_dist
    cost_comp_eql_p = solution_1_cost <= solution_2_cost
    dist_comp_p = solution_1_dist < solution_2_dist
    cost_comp_p = solution_1_cost < solution_2_cost

    dist_comp_eql_q = solution_1_dist >= solution_2_dist
    cost_comp_eql_q = solution_1_cost >= solution_2_cost
    dist_comp_q = solution_1_dist > solution_2_dist
    cost_comp_q = solution_1_cost > solution_2_cost

    if ((dist_comp_eql_p and cost_comp_eql_p) and (dist_comp_p or cost_comp_p)):
        return 1
    elif ((dist_comp_eql_q and cost_comp_eql_q) and (dist_comp_q or cost_comp_q)):
        return -1
    else:
        return 0
    

def FastNonDominatedSort(population):
    population_data = {}
    fronts = []
    solution_p = 0
    while solution_p < len(population):
        sols_dom_by_p = []
        dom_count_p = 0
        solution_q = 0
        while solution_q < len(population):
            dom_p_q = CheckDomination(population[solution_p], population[solution_q])
            # if p dominates q
            if dom_p_q == 1:
                sols_dom_by_p.append(solution_q)
            # if q dominates p
            elif dom_p_q == -1:
                dom_count_p += 1
            solution_q += 1
        population_data[solution_p] = {
            "dominates": sols_dom_by_p,
            "dominationCount": dom_count_p,
        }
        solution_p += 1
    while len(population_data):
        new_front = []
        for i in population_data:
            if population_data[i]["dominationCount"] == 0:
                new_front.append(i)
        for j in new_front:
            for submissive in population_data[j]["dominates"]:
                population_data[submissive]["dominationCount"] -= 1
            del population_data[j]
        fronts.append(new_front)
    return fronts

In [ ]:
def Evaluate(tour, matrix):
    value = 0
    i = 0
    while i < (len(tour)-1):
        value += matrix[tour[i]][tour[i+1]]
        i += 1
        pass
    value += matrix[tour[-1]][tour[0]]
    return value

def CrowdingDistance(front, population):
    distance = [0.0 for i in front]
    fitness_cost_values = [] 
    fitness_dist_values = []
    for i in front:
        fitness_cost_values.append(Evaluate(population[i], cost_matrix))
        fitness_dist_values.append(Evaluate(population[i], distance_matrix))
    cost_indices = sorted(
        range(len(fitness_cost_values)), 
        key=lambda k: fitness_cost_values[k],
        reverse=True
    )
    dist_indices = sorted(
        range(len(fitness_dist_values)), 
        key=lambda k: fitness_dist_values[k],
        reverse=True
    )
    # Should we take maximum and minimum fitness value possible of the current front
    # or the whole population?
    diff_cost_max = max(fitness_cost_values) - min(fitness_cost_values)
    diff_dist_max = max(fitness_dist_values) - min(fitness_dist_values)
    distance[cost_indices[0]] = math.inf
    distance[cost_indices[-1]] = math.inf
    i = 1
    while i < (len(front)-1):
        distance[cost_indices[i]] += (fitness_cost_values[cost_indices[i-1]] - fitness_cost_values[cost_indices[i+1]]) / diff_cost_max 
        i += 1
    distance[dist_indices[0]] = math.inf
    distance[dist_indices[-1]] = math.inf
    i = 1
    while i < (len(front)-1):
        distance[dist_indices[i]] += (fitness_dist_values[dist_indices[i-1]] - fitness_dist_values[dist_indices[i+1]]) / diff_dist_max 
        i += 1
    return distance

In [ ]:
def Selection(population):
    a, b = random.sample(range(len(population)), 2)
    parentA = population[a]
    parentB = population[b]
    return parentA, parentB

In [ ]:
def Crossover(parent1, parent2):
    if len(parent1) != len(parent2):
        return False
    else:
        child1 = [None for i in parent1]
        child2 = [None for i in parent2]

        # Generate 2 Random Points
        a, b = sorted(random.sample(range(len(parent1)+1), 2))

        # Indices to be Swapped
        unfilled_indinces = []
        for i in range(a):
            unfilled_indinces.append(i)
            pass
        for i in range(b, len(parent1)):
            unfilled_indinces.append(i)
            pass

        # Copy the Contents as it is
        for i in range(a, b):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
            pass

        # Swap from Parent2 to Child1
        ind = 0
        for i in parent2:
            if i not in child1:
                child1[unfilled_indinces[ind]] = i
                ind += 1
            pass

        # Swap from Parent1 to Child2
        ind = 0
        for i in parent1:
            if i not in child2:
                child2[unfilled_indinces[ind]] = i
                ind += 1
            pass
        return (child1, child2)

In [ ]:
def Mutation(chromosome):
    # Generate 2 Random Points
    a, b = sorted(random.sample(range(len(chromosome)), 2))
    chromosome[a], chromosome[b] = chromosome[b], chromosome[a]
    return chromosome

In [ ]:
def GenerateOffspring(parents):
    offspring = []
    while (len(offspring) != len(parents)):
        parentA, parentB = Selection(parents)
        childA, childB = Crossover(parentA, parentB)
        childA = Mutation(childA)
        childB = Mutation(childB)
        offspring.append(childA)
        offspring.append(childB)
    return offspring

In [ ]:
def GeneratePopulation(size, num_cities):
    permutations = []   
    for i in range(size):
        tour = [j for j in range(num_cities)]
        random.shuffle(tour)    
        permutations.append(tour)
        pass
    return permutations

def Evaluate(tour, matrix):
    value = 0
    i = 0
    while i < (len(tour)-1):
        value += matrix[tour[i]][tour[i+1]]
        i += 1
        pass
    value += matrix[tour[-1]][tour[0]]
    return value

if __name__ == '__main__':
    population_size = 20
    num_cities = 200
    num_generations = 500
    num_generations2 = 1000
    num_generations3 = 2000
    num_generations4 = 4000
    # Initial Population 
    parents = GeneratePopulation(population_size, num_cities)
    # Generate Children by Selection, Crossover and Mutation
    children = GenerateOffspring(parents)
    population = parents[:]
    # NewPopulation = Parents + Children
    population.extend(children)
    gen_num = 0
    while gen_num < num_generations:
        children = GenerateOffspring(parents)
        population = parents[:]
        # NewPopulation = Parents + Children
        population.extend(children)
        # print (gen_num, ":", len(population))
        # for i in population:
        #     print (i)
        fronts = FastNonDominatedSort(population)
#         for i in fronts:
#             print (i)
#         print ("--")
        new_parents = []
        for front in fronts:
            if (len(front) + len(new_parents)) <= population_size:
                for i in front:
                    # print (population[i])
                    new_parents.append(population[i])
            else:
                distance = CrowdingDistance(front, population)
                indices = sorted(
                    range(len(distance)), 
                    key=lambda k: distance[k],
                    reverse=True
                )
                i = 0
                while len(new_parents) != population_size:
                    new_parents.append(population[indices[i]])
                    i += 1
        parents = new_parents[:] 
        gen_num += 1
        pass
    function1 = []
    function2 = []
    # print ("Sol")
    for i in parents:
        # print (i)
        function1.append(Evaluate(i, distance_matrix))
        function2.append(Evaluate(i, cost_matrix))

        
    #_____________________________________________________________________#
    
    plt.figure()
    plt.scatter(function1, function2,color='blue' )   
    plt.scatter(objs[:,0],objs[:,1],color='red' )
#     plt.figure()
#     plt.plot(objs[:,0],objs[:,1],"ro")
    plt.show()

In [ ]:
if __name__ == '__main__':
    population_size = 20
    num_cities = 200
    num_generations = 500
    num_generations2 = 1000
    num_generations3 = 2000
    num_generations4 = 4000
    # Initial Population 
#_____________________________________________(2)_1000 step_________________________________#
    parents2 = GeneratePopulation(population_size, num_cities)
    # Generate Children by Selection, Crossover and Mutation
    children2 = GenerateOffspring(parents2)
    population2 = parents2[:]
    # NewPopulation = Parents + Children
    population2.extend(children2)
    gen_num2 = 0
    while gen_num2 < num_generations2:
        children2 = GenerateOffspring(parents2)
        population2 = parents2[:]
        # NewPopulation = Parents + Children
        population2.extend(children2)
        fronts2 = FastNonDominatedSort(population2)
#         for i in fronts:
#             print (i)
#         print ("--")
        new_parents2 = []
        for front in fronts2:
            if (len(front) + len(new_parents2)) <= population_size:
                for i in front:
                    # print (population[i])
                    new_parents2.append(population[i])
            else:
                distance = CrowdingDistance(front, population2)
                indices = sorted(
                    range(len(distance)), 
                    key=lambda k: distance[k],
                    reverse=True
                )
                i = 0
                while len(new_parents2) != population_size:
                    new_parents2.append(population2[indices[i]])
                    i += 1 
        parents2 = new_parents2[:] 
        gen_num2 += 1
        pass
    function3 = []
    function4 = []
    for i in parents2:
        function3.append(Evaluate(i, distance_matrix))
        function4.append(Evaluate(i, cost_matrix))
        
    #_____________________________________________________________________#
    
    plt.figure()
    plt.scatter(function3, function4,color='grey' )  
    plt.scatter(objs[:,0],objs[:,1],color='red' )
#     plt.figure()
#     plt.plot(objs[:,0],objs[:,1],"ro")
    plt.show()

In [ ]:
if __name__ == '__main__':
    population_size = 20
    num_cities = 200
    num_generations = 500
    num_generations2 = 1000
    num_generations3 = 2000
    num_generations4 = 4000
    # Initial Population 
        
#_____________________________________________(3)_ 2000 step_________________________________#
    parents3 = GeneratePopulation(population_size, num_cities)
    # Generate Children by Selection, Crossover and Mutation
    children3 = GenerateOffspring(parents3)
    population3 = parents3[:]
    # NewPopulation = Parents + Children
    population3.extend(children3)
    gen_num3 = 0
    while gen_num3 < num_generations3:
        children3 = GenerateOffspring(parents3)
        population3 = parents3[:]
        # NewPopulation = Parents + Children
        population3.extend(children3)
        fronts3 = FastNonDominatedSort(population3)
#         for i in fronts:
#             print (i)
#         print ("--")
        new_parents3 = []
        for front in fronts3:
            if (len(front) + len(new_parents3)) <= population_size:
                for i in front:
                    # print (population[i])
                    new_parents3.append(population[i])
            else:
                distance = CrowdingDistance(front, population3)
                indices = sorted(
                    range(len(distance)), 
                    key=lambda k: distance[k],
                    reverse=True
                )
                i = 0
                while len(new_parents3) != population_size:
                    new_parents3.append(population3[indices[i]])
                    i += 1 
        parents3 = new_parents3[:] 
        gen_num3 += 1
        pass
    function5 = []
    function6 = []
    for i in parents3:
        function5.append(Evaluate(i, distance_matrix))
        function6.append(Evaluate(i, cost_matrix))
        
    #_____________________________________________________________________#
    
    plt.figure()
    plt.scatter(function5, function6,color='aqua' )    
    plt.scatter(objs[:,0],objs[:,1],color='red' )
#     plt.figure()
#     plt.plot(objs[:,0],objs[:,1],"ro")
    plt.show()

In [ ]:
if __name__ == '__main__':
    population_size = 20
    num_cities = 200
    num_generations = 500
    num_generations2 = 1000
    num_generations3 = 2000
    num_generations4 = 4000
    # Initial Population 
    
    #_____________________________________________(4)4000 step__________________________________#
    parents4 = GeneratePopulation(population_size, num_cities)
    # Generate Children by Selection, Crossover and Mutation
    children4 = GenerateOffspring(parents4)
    population4 = parents4[:]
    # NewPopulation = Parents + Children
    population4.extend(children4)
    gen_num4 = 0
    while gen_num4 < num_generations4:
        children4 = GenerateOffspring(parents4)
        population4 = parents4[:]
        # NewPopulation = Parents + Children
        population4.extend(children4)
        fronts4 = FastNonDominatedSort(population4)
#         for i in fronts:
#             print (i)
#         print ("--")
        new_parents4 = []
        for front in fronts4:
            if (len(front) + len(new_parents4)) <= population_size:
                for i in front:
                    # print (population[i])
                    new_parents4.append(population[i])
            else:
                distance = CrowdingDistance(front, population4)
                indices = sorted(
                    range(len(distance)), 
                    key=lambda k: distance[k],
                    reverse=True
                )
                i = 0
                while len(new_parents4) != population_size:
                    new_parents4.append(population4[indices[i]])
                    i += 1 
        parents4 = new_parents4[:] 
        gen_num4 += 1
        pass
    function7 = []
    function8 = []
    for i in parents4:
        function7.append(Evaluate(i, distance_matrix))
        function8.append(Evaluate(i, cost_matrix))
        
    #_____________________________________________________________________#
    
    plt.figure()
    plt.scatter(function7, function8,color='yellow' )    
    plt.scatter(objs[:,0],objs[:,1],color='red' )
#     plt.figure()
#     plt.plot(objs[:,0],objs[:,1],"ro")
    plt.show()